In [ ]:
#importing necessary libs
import numpy as np
import pandas as pd 
import torch
import torch.nn as nn
import cv2
import os
import time
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [ ]:
#for using BERT
!pip install sentence_transformers

# **YOLO MODEL**

In [ ]:
#getting weights from pretrained model
weightsfile = '/kaggle/input/data-for-yolo-v3-kernel/yolov3.weights'
classfile = '/kaggle/input/data-for-yolo-v3-kernel/coco.names'
cfgfile = '/kaggle/working/yolov3.cfg'
sample_img1 = '/kaggle/working/input/insta20881.jpg'
input_dir = '/kaggle/working/input'
output_dir = '/kaggle/working/output'
nms_thesh = 0.5

In [ ]:
if not os.path.exists(input_dir):
    os.mkdir(input_dir)

In [ ]:
! wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
#! wget wget https://github.com/ayooshkathuria/pytorch-yolo-v3/raw/master/dog-cycle-car.png
! cp /kaggle/input/instagram-images-with-captions/instagram_data2/img2/insta20881.jpg /kaggle/working/input
#! cp /kaggle/input/data-for-yolo-v3-kernel/office.jpg /kaggle/working/input

We will create the model using the configuration file.First we will parse it and then feed to our model

In [ ]:
# Reading the file line by line (splitting on the ‘/n’ newline signal) 
# we store the parameters in a more readabale format within a dictionary, 
# and then transfer those to a list, we retturn that list as final_list.  
def parse_cfg(config_file):
    file = open(config_file,'r')
    file = file.read().split('\n')
    file =  [line for line in file if len(line)>0 and line[0] != '#']
    file = [line.lstrip().rstrip() for line in file]

    final_list = []
    element_dict = {}
    for line in file:

        if line[0] == '[':
            if len(element_dict) != 0:     # appending the dict stored on previous iteration
                    final_list.append(element_dict)
                    element_dict = {} # again emtying dict
            element_dict['type'] = ''.join([i for i in line if i != '[' and i != ']'])
            
        else:
            val = line.split('=')
            element_dict[val[0].rstrip()] = val[1].lstrip()  #removing spaces on left and right side
        
    final_list.append(element_dict) # appending the values stored for last set
    return final_list

In [ ]:
class DummyLayer(nn.Module):
    def __init__(self):
        super(DummyLayer, self).__init__()
        

        
class DetectionLayer(nn.Module):
    def __init__(self, anchors):
        super(DetectionLayer, self).__init__()
        self.anchors = anchors
        
        

def create_model(blocks):
#     blocks = parse_cfg(cfgfile)
    darknet_details = blocks[0]
    channels = 3 
    output_filters = []
    modulelist = nn.ModuleList()
    # we load he module list from pyorch’s nn.ModuleList and go on to enumerate through the parameter blocks. 
    # Of course, before any of that we declare a Sequential model. 
    # For each block, we check he block’s [‘type’] and if the block’s [type] is convolutional, upsample, root, yolo, as such 
    # then we intialize modules from nn.[Conv2d]/[Upsample] or, for type[‘route’] and type[‘shortcut’] both 
    # we respecitvely declare DummyLayer objects, all to be added to the sequential neural network module. 
    # For type[‘yolo’] layers we declare a DetectionLayer instance instead. 
    # The filters that need to be defined are also taken from the block parameters when needed within each type of module 
    #and finally we return  darknet_details and moduleList variables as output.
    
    for i,block in enumerate(blocks[1:]):
        seq = nn.Sequential()
        if (block["type"] == "convolutional"):
            activation = block["activation"]
            filters = int(block["filters"])
            kernel_size = int(block["size"])
            strides = int(block["stride"])
            use_bias= False if ("batch_normalize" in block) else True
            pad = (kernel_size - 1) // 2
            
            conv = nn.Conv2d(in_channels=channels, out_channels=filters, kernel_size=kernel_size, 
                             stride=strides, padding=pad, bias = use_bias)
            seq.add_module("conv_{0}".format(i), conv)
            
            if "batch_normalize" in block:
                bn = nn.BatchNorm2d(filters)
                seq.add_module("batch_norm_{0}".format(i), bn)

            if activation == "leaky":
                activn = nn.LeakyReLU(0.1, inplace = True)
                seq.add_module("leaky_{0}".format(i), activn)
            
        elif (block["type"] == "upsample"):
            upsample = nn.Upsample(scale_factor = 2, mode = "bilinear")
            seq.add_module("upsample_{}".format(i), upsample)
        
        elif (block["type"] == 'route'):
            # start and end is given in format (eg:-1 36 so we will find layer number from it.
            # we will find layer number in negative format
            # so that we can get the number of filters in that layer
            block['layers'] = block['layers'].split(',')
            block['layers'][0] = int(block['layers'][0])
            start = block['layers'][0]
            if len(block['layers']) == 1:               
                filters = output_filters[i + start]
                       
            
            elif len(block['layers']) > 1:
                block['layers'][1] = int(block['layers'][1]) - i 
                end = block['layers'][1]
                filters = output_filters[i + start] + output_filters[i + end]
                  
            
            route = DummyLayer()
            seq.add_module("route_{0}".format(i),route)
            # The route layer has an attribute ’layers’ which can have either one or two values. 
            # If the layers attribute has only one value, it outputs the feature maps of the layer indexed by its value. 
            # If the layers atribute has two values, it returns the concatenated feature maps of the layers indexed by its values, 
            # in which case we use torch.cat function wih he argument ‘1’ in order to concatenate the feature maps along their ‘depth’. 
      
      
        elif block["type"] == "shortcut":
            from_ = int(block["from"])
            shortcut = DummyLayer()
            seq.add_module("shortcut_{0}".format(i),shortcut)
            
            
        elif block["type"] == "yolo":
            mask = block["mask"].split(",")
            mask = [int(m) for m in mask]
            anchors = block["anchors"].split(",")
            anchors = [(int(anchors[i]), int(anchors[i + 1])) for i in range(0, len(anchors), 2)]
            anchors = [anchors[i] for i in mask]
            block["anchors"] = anchors
            
            detectorLayer = DetectionLayer(anchors)
            seq.add_module("Detection_{0}".format(i),detectorLayer)
                
        modulelist.append(seq)
        output_filters.append(filters)  
        channels = filters
    
    return darknet_details, modulelist
    

# Creating Yolo Model

In [ ]:
def prediction(x,inp_dim,anchors,num_classes,CUDA=False):
    # x --> 4D feature map
    batch_size = x.size(0)
    grid_size = x.size(2)
    stride =  inp_dim // x.size(2)   # factor by which current feature map reduced from input
    #grid_size = inp_dim // stride
    
    bbox_attrs = 5 + num_classes
    num_anchors = len(anchors)
    # 
    prediction = x.view(batch_size, bbox_attrs*num_anchors, grid_size*grid_size)
    prediction = prediction.transpose(1,2).contiguous()
    prediction = prediction.view(batch_size, grid_size*grid_size*num_anchors, bbox_attrs)
    
    # the dimension of anchors is wrt original image.We will make it corresponding to feature map
    anchors = [(a[0]/stride, a[1]/stride) for a in anchors]
    #Sigmoid the  centre_X, centre_Y. and object confidencce
    prediction[:,:,0] = torch.sigmoid(prediction[:,:,0])
    prediction[:,:,1] = torch.sigmoid(prediction[:,:,1])
    prediction[:,:,4] = torch.sigmoid(prediction[:,:,4])
    #Add the center offsets
    grid = np.arange(grid_size)
    a,b = np.meshgrid(grid, grid)

    x_offset = torch.FloatTensor(a).view(-1,1) #(1,gridsize*gridsize,1)
    y_offset = torch.FloatTensor(b).view(-1,1)

    if CUDA:
        x_offset = x_offset.cuda()
        y_offset = y_offset.cuda()

    x_y_offset = torch.cat((x_offset, y_offset), 1).repeat(1,num_anchors).view(-1,2).unsqueeze(0)
    

    prediction[:,:,:2] += x_y_offset

    #log space transform height and the width
    anchors = torch.FloatTensor(anchors)

    if CUDA:
        anchors = anchors.cuda()

    anchors = anchors.repeat(grid_size*grid_size, 1).unsqueeze(0)
    prediction[:,:,2:4] = torch.exp(prediction[:,:,2:4])*anchors #width and height
    prediction[:,:,5: 5 + num_classes] = torch.sigmoid((prediction[:,:, 5 : 5 + num_classes]))    
    prediction[:,:,:4] *= stride    
    return prediction
    

In [ ]:
class Darknet(nn.Module):
    def __init__(self, cfgfile):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(cfgfile)
        self.net_info, self.module_list = create_model(self.blocks)
        
    def forward(self, x, CUDA=False):
        '''
        We start by defining the modules, outputs and write variables. 
        When defining modules, we iterate over self.blocks[1:] rather than self.blocks since the first element of self.blocks is a ‘net’ block 
        which isn't strictly a component of a forward pass. 
        Then we iterate over the modules in the nework using modüle_list, which holds their order identical to the one in the configuration file. 
        So, simplified as everything is, we just run the input through the layers as they come. 
        The convolutional or upsample modules are as straightforward as that, 
        meanwhile route layer instances need to check for two conditions. 
        '''
        modules = self.blocks[1:]
        outputs = {}  
        # The outputs dictionary will be used to cache the outputs maps from previous layers as the route and shortcu layers need them. 
        # The key value pairs are respectively made up of the indices of the layers and the values are the corresponding feature maps. 
        
        write = 0     #This is explained 
        #As for the yolo modüle that declares detection layers, its ouput is predictably a convolutional feature map 
        # that has the bounding box attributes along the depth of the feature map. 
        # We transform our outputs tensors in order to have them share spatial dimensions; 
        # then we would need to concatenate the three detection maps into one tensor. 
        #We don’t initialize a concatenation tensor yet and check whether it has been or not with the ‘write’ flag: 
        #If i’s equal o zero then it hasn’t been initialized yet and if it is 1 then the first detection has happened already 
        # and so we have a tensor we can concatenate the rest of the maps to. 
        # This rounabout way o create a tensor to hold the maps together is 
        # the way i is because of our inability to initialize an empty tensor to be populated later.
        
        for i, module in enumerate(modules):        
            module_type = (module["type"])
            if module_type == "convolutional" or module_type == "upsample":
                x = self.module_list[i](x)
                outputs[i] = x
                
            elif module_type == "route":
                layers = module["layers"]
                layers = [int(a) for a in layers]
                if len(layers) == 1:
                    x = outputs[i + layers[0]]
                if len(layers) > 1:
                    map1 = outputs[i + layers[0]]
                    map2 = outputs[i + layers[1]]
                    x = torch.cat((map1,map2),1)
       
                outputs[i] = x
                
            elif  module_type == "shortcut":
                from_ = int(module["from"])

                # just adding outputs for residual network
                x = outputs[i-1] + outputs[i+from_]  
                outputs[i] = x
                
            elif module_type == 'yolo':
                anchors = self.module_list[i][0].anchors
                
                #Get the input dimensions
                inp_dim = int(self.net_info["height"])
                #Get the number of classes
                num_classes = int(module["classes"])
            
                #Transform 
                x = x.data   # get the data at that point
                x = prediction(x,inp_dim,anchors,num_classes)
                
                if not write:              #if no collector has been intialised. 
                    detections = x
                    write = 1
                else:       
                    detections = torch.cat((detections, x), 1)

                outputs[i] = outputs[i-1]
                
        try:
            return detections   #return detections if present
        except:
            return 0
    #The factors to consider here are firstly, that the weights can only belong to eiher a batch norm or a convolutional type layer,
    #secondly, that those weights are stored in the same order of the layers in the configuration file, and 
    # thirdly that when a batch norm layer appears within a ‘convolutional’ block there aren’t any biases. 
    # Conversely, if there are no batch layers in that block then there will be biases. 
    # We read the header and the actual weights separately according to the file’s structure and move on to extracing the weights 
    # then loading them into their lawful layers. 
    # The variable ‘ptr’ which is basically a pointer points out where we are in the weights array. 
    # It bears mentioning that according to whether there is a batch norm layer ina convolutional block or not 
    # we read the weights in different ways.
    def load_weights(self, weightfile):
        
        #Open the weights file
        fp = open(weightfile, "rb")

        #The first 4 values are header information 
        # 1. Major version number
        # 2. Minor Version Number
        # 3. Subversion number 
        # 4. IMages seen 
        header = np.fromfile(fp, dtype = np.int32, count = 5)
        self.header = torch.from_numpy(header)
        self.seen = self.header[3]
        
        #The rest of the values are the weights
        # Let's load them up
        weights = np.fromfile(fp, dtype = np.float32)
        
        ptr = 0
        for i in range(len(self.module_list)):
            module_type = self.blocks[i + 1]["type"]
            
            if module_type == "convolutional":
                model = self.module_list[i]
                try:
                    batch_normalize = int(self.blocks[i+1]["batch_normalize"])
                except:
                    batch_normalize = 0
                
                conv = model[0]
                
                if (batch_normalize):
                    bn = model[1]
                    
                    #Get the number of weights of Batch Norm Layer
                    num_bn_biases = bn.bias.numel()
                    
                    #Load the weights
                    bn_biases = torch.from_numpy(weights[ptr:ptr + num_bn_biases])
                    ptr += num_bn_biases
                    
                    bn_weights = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    bn_running_mean = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    bn_running_var = torch.from_numpy(weights[ptr: ptr + num_bn_biases])
                    ptr  += num_bn_biases
                    
                    #Cast the loaded weights into dims of model weights. 
                    bn_biases = bn_biases.view_as(bn.bias.data)
                    bn_weights = bn_weights.view_as(bn.weight.data)
                    bn_running_mean = bn_running_mean.view_as(bn.running_mean)
                    bn_running_var = bn_running_var.view_as(bn.running_var)

                    #Copy the data to model
                    bn.bias.data.copy_(bn_biases)
                    bn.weight.data.copy_(bn_weights)
                    bn.running_mean.copy_(bn_running_mean)
                    bn.running_var.copy_(bn_running_var)
                
                else:
                    #Number of biases
                    num_biases = conv.bias.numel()
                
                    #Load the weights
                    conv_biases = torch.from_numpy(weights[ptr: ptr + num_biases])
                    ptr = ptr + num_biases
                    
                    #reshape the loaded weights according to the dims of the model weights
                    conv_biases = conv_biases.view_as(conv.bias.data)
                    
                    #Finally copy the data
                    conv.bias.data.copy_(conv_biases)
                    
                    
                #Let us load the weights for the Convolutional layers
                num_weights = conv.weight.numel()
                
                #Do the same as above for weights
                conv_weights = torch.from_numpy(weights[ptr:ptr+num_weights])
                ptr = ptr + num_weights

                conv_weights = conv_weights.view_as(conv.weight.data)
                conv.weight.data.copy_(conv_weights)
                # Note: we dont have bias for conv when batch normalization is there

# YOLO Predictions 

In [ ]:
def bbox_iou(box1, box2):
    """
  
    Bbox_iou is one that deals wih bounding box Intersection over Union calculation. 
    Basically, if we have two bounding boxes of the same class that have IoU larger than a certain threshold value 
    then the one with the lower classificaion confidence is discarded.  
    In the loop within the function body if any of all the bounding boxes with a larger index than ‘i’ 
    has an ‘iou’ larger than ‘nms_thresh’ (our threshold) then that box is discarded.  
    
  
    
    """
    #Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]
    
    #get the corrdinates of the intersection rectangle
    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)
    
    #Intersection area
    inter_area = torch.clamp(inter_rect_x2 - inter_rect_x1 + 1, min=0) * torch.clamp(inter_rect_y2 - inter_rect_y1 + 1, min=0)
 
    #Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou


def unique(tensor):
    '''
    ’unique’ lets us count the overall unique class values when muliple ‘true’ detecions of the same class is present in some image. 
    This is accomplished through NMS. 
    '''
    tensor_np = tensor.cpu().numpy()
    unique_np = np.unique(tensor_np)
    unique_tensor = torch.from_numpy(unique_np)
    
    tensor_res = tensor.new(unique_tensor.shape)
    tensor_res.copy_(unique_tensor)
    return tensor_res

#TBelow, objectness confidence thresholding, sets a threshold so that we can discard the bounding boxes 
# that aren’t made in confident classification. 
# The bounding boxes we are left with are still desribed by their center coordinates plus their height and width 
# but for IoU calculation we are better off with these boxes’ diagonals. 
# Thus we tranform box_corner as necessary. 
# And we can have several instances ‘true’ for a single class within any given image so 
# this confidence thresholding and NMS have to be done for an image at once, 
# the operations involved in the process unable to be vectorised.   

#As an aside, It’s worth noting here that within this function body the write flag represents the presence of the ‘output’ tensor. 
def write_results(prediction, confidence, num_classes, nms_conf = 0.4):
    # taking only values above a particular threshold and set rest everything to zero
    conf_mask = (prediction[:,:,4] > confidence).float().unsqueeze(2)
    prediction = prediction*conf_mask
    
    
    #(center x, center y, height, width) attributes of our boxes, 
    #to (top-left corner x, top-left corner y, right-bottom corner x, right-bottom corner y)
    box_corner = prediction.new(prediction.shape)
    box_corner[:,:,0] = (prediction[:,:,0] - prediction[:,:,2]/2)
    box_corner[:,:,1] = (prediction[:,:,1] - prediction[:,:,3]/2)
    box_corner[:,:,2] = (prediction[:,:,0] + prediction[:,:,2]/2) 
    box_corner[:,:,3] = (prediction[:,:,1] + prediction[:,:,3]/2)
    prediction[:,:,:4] = box_corner[:,:,:4]
    
    batch_size = prediction.size(0)
    write = False
    
    # we can do non max suppression only on individual images so we will loop through images
    for ind in range(batch_size):  
        image_pred = prediction[ind] 
        # we will take only those rows with maximm class probability
        # and corresponding index
        max_conf, max_conf_score = torch.max(image_pred[:,5:5+ num_classes], 1)
        max_conf = max_conf.float().unsqueeze(1)
        max_conf_score = max_conf_score.float().unsqueeze(1)
        seq = (image_pred[:,:5], max_conf, max_conf_score)
        # concatinating index values and max probability with box cordinates as columns
        image_pred = torch.cat(seq, 1) 
        #Remember we had set the bounding box rows having a object confidence
        # less than the threshold to zero? Let's get rid of them.
        non_zero_ind =  (torch.nonzero(image_pred[:,4])) # non_zero_ind will give the indexes 
        image_pred_ = image_pred[non_zero_ind.squeeze(),:].view(-1,7)
        try:
            #Get the various classes detected in the image
            img_classes = unique(image_pred_[:,-1]) # -1 index holds the class index
        except:
             continue
       
        # As each box has 85 attributes, 80 of which are class scores, 
    # we are mostly trying to ensure that we are dealing with the maximum value out of all those 80 as the box’s class. 
    # What we do here is that we remove those 80 attribues we won’t be needing but 
    # add the index of the class that has the maximum value alongside its actual confidence score. 

    

        #Having fetched the unique classes as ‘cls’ in an image, we perform NMS for each of those classes. 
        for cls in img_classes:
            #perform NMS
            #get the detections with one particular class
            cls_mask = image_pred_*(image_pred_[:,-1] == cls).float().unsqueeze(1)
            class_mask_ind = torch.nonzero(cls_mask[:,-2]).squeeze()
            image_pred_class = image_pred_[class_mask_ind].view(-1,7)
            
            # sort them based on probability
            conf_sort_index = torch.sort(image_pred_class[:,4], descending = True )[1]#getting index
            image_pred_class = image_pred_class[conf_sort_index]
            idx = image_pred_class.size(0)
            
            for i in range(idx):
                #Get the IOUs of all boxes that come after the one we are looking at 
                 #in the loop
                try:
                    ious = bbox_iou(image_pred_class[i].unsqueeze(0), image_pred_class[i+1:])
                except ValueError:
                    break
                except IndexError:
                    break
                
                #Zero out all the detections that have IoU > treshhold
                iou_mask = (ious < nms_conf).float().unsqueeze(1)
                image_pred_class[i+1:] *= iou_mask
                
                #Remove the non-zero entries
                non_zero_ind = torch.nonzero(image_pred_class[:,4]).squeeze()
                image_pred_class = image_pred_class[non_zero_ind].view(-1,7)
          
            #Concatenate the batch_id of the image to the detection
            #this helps us identify which image does the detection correspond to 
            #We use a linear straucture to hold ALL the detections from the batch
            #the batch_dim is flattened
            #batch is identified by extra batch column
            
            #creating a row with index of images
            batch_ind = image_pred_class.new(image_pred_class.size(0), 1).fill_(ind)
            seq = batch_ind, image_pred_class
            if not write:
                output = torch.cat(seq,1)
                write = True
            else:
                out = torch.cat(seq,1)
                output = torch.cat((output,out))
    
    return output
            

In [ ]:
# Utility functions

# function to load the classes
def load_classes(namesfile):
    '''outputs a dicionary wthat has its keys as class indices and their names as strings as values. '''
    fp = open(namesfile, "r")
    names = fp.read().split("\n")[:-1]
    return names

# function converting images from opencv format to torch format
def prep_image(img, inp_dim):

    """
    we move on to define the prep_image function to transform our image 
    (that’s loaded with opencv wih BGR as its order of channels) 
    into PyTorch’s image input format, which is BatchesxChannelsxHeightxWidth.
    """

    orig_im = cv2.imread(img)
    dim = orig_im.shape[1], orig_im.shape[0]
    img = (letterbox_image(orig_im, (inp_dim, inp_dim)))
    img_ = img[:,:,::-1].transpose((2,0,1)).copy()
    img_ = torch.from_numpy(img_).float().div(255.0).unsqueeze(0)
    return img_, orig_im, dim

#function letterbox_image that resizes our image, keeping the 
# aspect ratio consistent, and padding the left out areas with the color (128,128,128)
def letterbox_image(img, inp_dim):
    '''Right after we also the letterbox_image function which resizes our image while no touching the  aspect ratio 
    and padding the left out areas with the color ‘grey’, which is also (128, 128, 128). '''
    img_w, img_h = img.shape[1], img.shape[0]
    w, h = inp_dim
    new_w = int(img_w * min(w/img_w, h/img_h))
    new_h = int(img_h * min(w/img_w, h/img_h))
    resized_image = cv2.resize(img, (new_w,new_h), interpolation = cv2.INTER_CUBIC)
    
    canvas = np.full((inp_dim[1], inp_dim[0], 3), 128)

    canvas[(h-new_h)//2:(h-new_h)//2 + new_h,(w-new_w)//2:(w-new_w)//2 + new_w,:] = resized_image
    
    return canvas

In [ ]:
CUDA = False
batch_size = 2
#Set up the neural network
print("Loading network.....")
model = Darknet(cfgfile)
model.load_weights(weightsfile)
print("Network successfully loaded")
classes = load_classes(classfile)
print('Classes loaded')
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32
query=''

#If there's a GPU availible, put the model on GPU
if CUDA:
    model.cuda()

#Set the model in evaluation mode
model.eval()


# read images from folder 'images' or direcly  image
read_dir = time.time()
#Detection phase
try:
    imlist = [os.path.join(os.path.realpath('.'), input_dir, img) for img in os.listdir(input_dir)]
except NotADirectoryError:
    imlist = []
    imlist.append(os.path.join(os.path.realpath('.'), input_dir))
except FileNotFoundError:
    print ("No file or directory with the name {}".format(input_dir))
    exit()
    
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
load_batch = time.time()

# preparing list of loaded images
# [[image,original_image,dim[0],dim[1]]]
batches = list(map(prep_image, imlist, [inp_dim for x in range(len(imlist))]))
im_batches = [x[0] for x in batches] # list of resized images
orig_ims = [x[1] for x in batches] # list of original images
im_dim_list = [x[2] for x in batches] # dimension list
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2) #repeating twice
    
    
if CUDA:
    im_dim_list = im_dim_list.cuda()

    
# converting image to batches    
reminder = 0
if (len(im_dim_list) % batch_size): #if reminder is there, reminder = 1
    reminder = 1

if batch_size != 1:
    num_batches = len(imlist) // batch_size + reminder            
    im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,len(im_batches))])) 
                 for i in range(num_batches)] 
    
    
i = 0
write = False
    
objs = {}    

#In the loop concerned wih detection below we iterate over the batches, 
# generate a prediction and concatenate the prediction tensors of all the images into the tensor ‘output’. 
# For a batch, we time the detection duration. 
# In the outputs of write_prediction was also the index of an image within the batch, we take that index and transform it with 
# ‘prediction[:,0] += i*batch_size’ so that i represents the index of an image in imlist instead.  

for batch in im_batches:
        #load the image 
        start = time.time()
        if CUDA:
            batch = batch.cuda()       
        #Apply offsets to the result predictions
        #Tranform the predictions as described in the YOLO paper
        #flatten the prediction vector 
        # B x (bbox cord x no. of anchors) x grid_w x grid_h --> B x bbox x (all the boxes) 
        # Put every proposed box as a row.
        with torch.no_grad():
            prediction = model(Variable(batch), CUDA)
        
        prediction = write_results(prediction, confidence=0.5, num_classes=80, nms_conf = nms_thesh)
        
        if type(prediction) == int:
            i += 1
            continue


        prediction[:,0] += i*batch_size
                  
        if not write:
            output = prediction
            write = 1
        else:
            output = torch.cat((output,prediction))  # concating predictions from each batch
        i += 1
        
        if CUDA:
            torch.cuda.synchronize()
    
try:
    output
except NameError:
    print("No detections were made")
    exit()

    
#Before we draw the bounding boxes, the predictions contained in our output tensor 
#are predictions on the padded image, and not the original image. Merely, re-scaling them 
#to the dimensions of the input image won't work here. We first need to transform the
#co-ordinates of the boxes to be measured with respect to boundaries of the area on the
#padded image that contains the original image


im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())
scaling_factor = torch.min(inp_dim/im_dim_list,1)[0].view(-1,1)
output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2
output[:,1:5] /= scaling_factor
    
for i in range(output.shape[0]):
    output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
    output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])
    
def getLabels(labels):
    global query
    query=labels
    

def write(x, batches, results):
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    color = (0,0,255)
    cv2.rectangle(img, c1, c2,color, 2)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1)
    getLabels(label)
    return img
    
            
list(map(lambda x: write(x, im_batches, orig_ims), output))
      
det_names = pd.Series(imlist).apply(lambda x: "{}/det_{}".format(output_dir,x.split("/")[-1]))
    
list(map(cv2.imwrite, det_names, orig_ims))

    

torch.cuda.empty_cache()
    

In [ ]:
img = cv2.imread('/kaggle/working/output/det_insta20881.jpg') 
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,10))
plt.imshow(img)

# **BERT MODEL**

In [ ]:
from sentence_transformers import *

In [ ]:
#getting the model
model=SentenceTransformer('bert-large-nli-mean-tokens')

In [ ]:
#getting the dataset for fine tuning
train=pd.read_csv('../input/instagram-images-with-captions/instagram_data/captions_csv.csv',delimiter=',')
train['Caption']=train['Caption'].astype(str)

In [ ]:
#creating embeddings from the model word corpus
embeddings = model.encode(train['Caption'])

In [ ]:
#printing the label generated from yolo
print(query)

In [ ]:
#encoding the query 
query_embedding = model.encode(query)

In [ ]:
print(query_embedding)

# RESULT
> Printing the caption SUGGESTIONS

In [ ]:
top_k=5
cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
cos_scores = cos_scores.cpu()

#We use torch.topk to find the highest 5 scores for similiarty
top_results = torch.topk(cos_scores, k=top_k)

print("\nTop 5 most similar sentences in corpus:")
image_id=[]
for score, idx in zip(top_results[0], top_results[1]):
    print(train['Caption'].values[idx])


### Reference:

https://arxiv.org/pdf/1804.02767.pdf <br>
https://github.com/ayooshkathuria/pytorch-yolo-v3/blob/master/util.py <br>

